In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv("amazon_co_ecommerce_sample.csv")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df[["price","number_available_in_stock"]].to_csv("amzn-tmp1.csv")

From "amzn-tmp1.csv" to "amzn-tmp1-bereinigt.csv" the pound sign in "price" has to be removed and "number_available_in_stock" has to be split, yielding a number and a type or two blanks

This operation has been performed outside this script. In a later version it should be done within the script

In [2]:
df1 = pd.read_csv("amzn-tmp1-bereinigt.csv", index_col=0, thousands=',')

In [3]:
df1['ProdIdA'] = np.random.randint(1, 999, df1.shape[0])
df1['ProdIdB'] = np.random.randint(1, 99999, df1.shape[0])

In [4]:
df1.head()

,price,number_available_in_stock,type,ProdIdA,ProdIdB
ord,,,,,
0,3.42,5,new,913,66011
1,16.99,0,NaN,498,12692
2,9.99,2,new,321,28576
3,39.99,0,NaN,574,98874
4,32.19,0,NaN,270,70449


In [5]:
def printable_char(nmb):
    # fancy function that converts certain integers to printable characters
    x = nmb % 64
    c = "prd-"
    if x < 64:
        x = x+64
    if x == 64 or x>90 and x<97 or x>=123:
        c = c+str(x)
    else:
        c = c+chr(int(str(x)))
    return c

In [6]:
def to_number(x):
    if pd.isna(x):
        return 0
    if type(x) == str:
        if re.match(r'.* - ',x):
            x = x.split(" - ")[0]
        if re.match(r'[0-9]+,.*',x):
            x = x.replace(',','')
        return float(x)
    else:
        return x

In [7]:
def categorize(a):
    if a>0 and a<=1:
        return "O"
    elif a>1 and a<=5:
        return "A"
    elif a>5 and a<=10:
        return "B"
    elif a>10 and a<=50:
        return "C"
    elif a>50 and a<=100:
        return "D"
    elif a>100:
        return "E"
    else:
         return "not set"

In [8]:
df1['price'] = df1['price'].apply(lambda x: to_number(x))

In [9]:
df1['ProdCateg'] = df1['ProdIdA'].apply(lambda x: printable_char(x))

In [10]:
df1['PriceCateg'] = df1['price'].apply(lambda x: categorize(x))

In [11]:
df1.dtypes

price                        float64
number_available_in_stock     object
type                          object
ProdIdA                        int64
ProdIdB                        int64
ProdCateg                     object
PriceCateg                    object
dtype: object

In [12]:
df1.head()

,price,number_available_in_stock,type,ProdIdA,ProdIdB,ProdCateg,PriceCateg
ord,,,,,,,
0,3.42,5,new,913,66011,prd-Q,A
1,16.99,0,NaN,498,12692,prd-r,C
2,9.99,2,new,321,28576,prd-A,B
3,39.99,0,NaN,574,98874,prd-126,C
4,32.19,0,NaN,270,70449,prd-N,C


Filtering 'ProdCateg' with numerals

In [13]:
df1[df1['ProdCateg'].str.contains('prd-[0-9]+', regex=True)].head()

,price,number_available_in_stock,type,ProdIdA,ProdIdB,ProdCateg,PriceCateg
ord,,,,,,,
3,39.99,0,NaN,574,98874,prd-126,C
7,69.93,3,new,191,54273,prd-127,D
33,0.00,0,NaN,576,74984,prd-64,not set
41,50.81,16,new,29,3176,prd-93,D
45,11.58,31,new,62,27403,prd-126,C


In [14]:
df1.to_csv("inventory.csv")